In [1]:
# pip install transformers datasets scikit-learn torch

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("datasets/cleanedDataSecondModel_merged.csv")
#remame
df = df[['quote', 'general_label_encoded']]
df.columns = ['text', 'label']

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [3]:
from transformers import DistilBertTokenizerFast

#tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


In [4]:
import torch

class QuoteDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx])
            for key, val in self.encodings.items()
        } | {'labels': torch.tensor(self.labels[idx])}

train_dataset = QuoteDataset(train_encodings, train_labels)
test_dataset = QuoteDataset(test_encodings, test_labels)


In [7]:
# pip install transformers[torch]


Note: you may need to restart the kernel to use updated packages.


In [5]:
# pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

num_labels = len(set(df['label']))
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs",
    logging_steps=10
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\VJ24-1\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:401: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss
1,1.247700,1.244463
2,0.723000,1.110533
3,0.532400,1.082193
4,0.244300,1.171032


TrainOutput(global_step=400, training_loss=0.7888636872172355, metrics={'train_runtime': 293.4384, 'train_samples_per_second': 10.905, 'train_steps_per_second': 1.363, 'total_flos': 87765911884800.0, 'train_loss': 0.7888636872172355, 'epoch': 4.0})

In [6]:
import numpy as np
from sklearn.metrics import classification_report

preds = trainer.predict(test_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)

print(classification_report(test_labels, pred_labels))


              precision    recall  f1-score   support

           0       0.86      0.40      0.55        15
           1       0.62      0.62      0.62         8
           2       0.48      0.50      0.49        42
           3       0.64      0.55      0.59        29
           4       0.79      0.76      0.77        49
           5       0.61      0.74      0.67        57

    accuracy                           0.64       200
   macro avg       0.67      0.59      0.61       200
weighted avg       0.65      0.64      0.63       200



In [11]:
#predict
test_quote = "Success is not final, failure is not fatal"
inputs = tokenizer(test_quote, return_tensors="pt", truncation=True, padding=True, max_length=128)

device = next(model.parameters()).device 
inputs = {key: val.to(device) for key, val in inputs.items()}

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

print(f"\nPredicted label for: \"{test_quote}\" → {predicted_label}")



Predicted label for: "Success is not final, failure is not fatal" → 5


5 -> Self & Inner Growth